In [1]:
# Imports
import pandas as pd
import seaborn as sns
import networkx as net
import freeman as fm
import datetime as dt
from dateutil.relativedelta import relativedelta

In [2]:
df = pd.read_csv('HRDataset_v14.csv')
df.head(10)

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,...,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,...,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,...,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,...,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,...,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
5,"Anderson, Linda",10002,0,0,0,1,5,4,0,57568,...,Amy Dunn,11.0,LinkedIn,Exceeds,5.00,5,0,1/7/2019,0,15
6,"Andreola, Colby",10194,0,0,0,1,4,3,0,95660,...,Alex Sweetwater,10.0,LinkedIn,Fully Meets,3.04,3,4,1/2/2019,0,19
7,"Athwal, Sam",10062,0,4,1,1,5,3,0,59365,...,Ketsia Liebig,19.0,Employee Referral,Fully Meets,5.00,4,0,2/25/2019,0,19
8,"Bachiochi, Linda",10114,0,0,0,3,5,3,1,47837,...,Brannon Miller,12.0,Diversity Job Fair,Fully Meets,4.46,3,0,1/25/2019,0,4
9,"Bacong, Alejandro",10250,0,2,1,1,3,3,0,50178,...,Peter Monroe,7.0,Indeed,Fully Meets,5.00,5,6,2/18/2019,0,16


In [3]:
# Interpretando as colunas do CSV
cols = df.columns.to_list()

for i in range(len(cols)):
    print(f"Column {i}: {cols[i]}")

Column 0: Employee_Name
Column 1: EmpID
Column 2: MarriedID
Column 3: MaritalStatusID
Column 4: GenderID
Column 5: EmpStatusID
Column 6: DeptID
Column 7: PerfScoreID
Column 8: FromDiversityJobFairID
Column 9: Salary
Column 10: Termd
Column 11: PositionID
Column 12: Position
Column 13: State
Column 14: Zip
Column 15: DOB
Column 16: Sex
Column 17: MaritalDesc
Column 18: CitizenDesc
Column 19: HispanicLatino
Column 20: RaceDesc
Column 21: DateofHire
Column 22: DateofTermination
Column 23: TermReason
Column 24: EmploymentStatus
Column 25: Department
Column 26: ManagerName
Column 27: ManagerID
Column 28: RecruitmentSource
Column 29: PerformanceScore
Column 30: EngagementSurvey
Column 31: EmpSatisfaction
Column 32: SpecialProjectsCount
Column 33: LastPerformanceReview_Date
Column 34: DaysLateLast30
Column 35: Absences


In [4]:
# Filtrando algumas colunas + valores que nao servem para analise
filters = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35]
cols_to_filter = [df.columns[i] for i in filters]

df1 = df.drop(cols_to_filter, axis=1)
df1 = df1.drop(df1.loc[df1['EmploymentStatus'] == 'Voluntarily Terminated'].index)
df1.reset_index(drop=True, inplace=True)

# Ajuste no DOB para ser datetime
df1['DOB'] = df1['DOB'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%y'))

df1.head(10)

,EmpID,PerfScoreID,DOB,Sex,RaceDesc,EmploymentStatus,ManagerName
0,10026,4,1983-07-10,M,White,Active,Michael Albert
1,10088,3,1988-09-27,F,White,Active,Elijiah Gray
2,10002,4,1977-05-22,F,White,Active,Amy Dunn
3,10194,3,1979-05-24,F,White,Active,Alex Sweetwater
4,10062,3,1983-02-18,M,White,Active,Ketsia Liebig
5,10114,3,1970-02-11,F,Black or African American,Active,Brannon Miller
6,10250,3,1988-01-07,M,White,Active,Peter Monroe
7,10012,4,1988-07-04,M,Black or African American,Active,Simon Roup
8,10265,3,1983-07-20,M,Two or more races,Active,Kelley Spirea
9,10061,3,1981-10-18,M,White,Terminated for Cause,Kelley Spirea


In [6]:
# Versao final do dataframe possui as informacoes do anterior mas altera as variaveis categoricas em numericas
data = pd.DataFrame() # Novo dataframe para poder mudar os nomes das colunas

date_uploaded = dt.datetime(2020, 10, 19)

data['ID_Funcionario'] = df1['EmpID']
data['Idade'] = df1['DOB'].apply(lambda x: relativedelta(date_uploaded, x).years)
data['Genero'] = df1['Sex'].apply(lambda x: 1 if x == 'F' else 0)
data['Performance'] = df1['PerfScoreID']
data['Raca'] = df1['RaceDesc'].apply(lambda x : list(df1['RaceDesc'].unique()).index(x))
data['Gerente'] = df1['ManagerName'].apply(lambda x : list(df1['ManagerName'].unique()).index(x))
data['Demitido'] = df1['EmploymentStatus'].apply(lambda x: 0 if x == 'Active' else 1)

data.head(10)

,ID_Funcionario,Idade,Genero,Performance,Raca,Gerente,Demitido
0,10026,37,0,4,0,0,0
1,10088,32,1,3,0,1,0
2,10002,43,1,4,0,2,0
3,10194,41,1,3,0,3,0
4,10062,37,0,3,0,4,0
5,10114,50,1,3,1,5,0
6,10250,32,0,3,0,6,0
7,10012,32,0,4,1,7,0
8,10265,37,0,3,2,8,0
9,10061,39,0,3,0,8,1


### Analise dos Dados

In [9]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(data.drop('Demitido', axis=1), data['Demitido'], test_size=0.30, random_state=101)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression()

In [14]:
# Analisando resultados
from sklearn.metrics import classification_report

predictions = logmodel.predict(X_test)
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        62
           1       0.00      0.00      0.00         5

    accuracy                           0.93        67
   macro avg       0.46      0.50      0.48        67
weighted avg       0.86      0.93      0.89        67



In [18]:
import statsmodels.discrete.discrete_model as sm

logit = sm.Logit(data['Demitido'], data[['Idade'] + ['Genero'] + ['Performance'] + ['Raca']])
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.243578
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Demitido   No. Observations:                  223
Model:                          Logit   Df Residuals:                      219
Method:                           MLE   Df Model:                            3
Date:                Wed, 04 Nov 2020   Pseudo R-squ.:                 0.05641
Time:                        23:00:48   Log-Likelihood:                -54.318
converged:                       True   LL-Null:                       -57.565
Covariance Type:            nonrobust   LLR p-value:                   0.08989
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Idade           0.0113      0.012      0.943      0.346      -0.012       0.035
Genero          0.1031      0.524      0.197      0.844      -0.923       1.130
Performance    -0.9966      0.204     -4.894      0.000      -1.396      -0.597
Raca           -0.2207      0.306     -0.720      0.471      -0.821       0.380
===============================================================================
"""